In [1]:
import random
import torch
from torch import nn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import gymnasium as gym
from torch.distributions.normal import Normal
from tqdm import tqdm
from tqdm import trange
from maglev_env import MagneticEnv, DT

In [2]:
class Policy_Network(nn.Module):
    """Parametrized Policy Network."""

    def __init__(self, obs_space_dims: int, action_space_dims: int):
        """Initializes a neural network that estimates the mean and standard deviation
         of a normal distribution from which an action is sampled from.

        Args:
            obs_space_dims: Dimension of the observation space
            action_space_dims: Dimension of the action space
        """
        super().__init__()

        # NOTE think more about these values
        hidden_space1 = 128
        hidden_space2 = 128
        hidden_space3 = 128

        # Shared Network
        self.shared_net = nn.Sequential(
            nn.Linear(obs_space_dims, hidden_space1),
            nn.ReLU(),
            nn.Linear(hidden_space1, hidden_space2),
            nn.ReLU(),
            nn.Linear(hidden_space2, hidden_space3),
            nn.ReLU(),
        )

        # Policy Mean specific Linear Layer
        self.policy_mean_net = nn.Sequential(
            nn.Linear(hidden_space3, action_space_dims)
        )

        # Policy Std Dev specific Linear Layer
        # NOTE do we want relu on this?
        self.policy_stddev_net = nn.Sequential(
            nn.Linear(hidden_space3, action_space_dims),
            nn.ReLU()
        )


    def forward(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        """Conditioned on the observation, returns the mean and standard deviation
         for each normal distribution from which an action is sampled from.

        Args:
            x: Observation from the environment

        Returns:
            action_means: predicted means of the action space's normal distribution
            action_stddevs: predicted standard deviation of the action space's normal distribution
        """
        shared_features = self.shared_net(x.float())

        action_means = self.policy_mean_net(shared_features)
        action_stddevs = torch.log(
            1 + torch.exp(self.policy_stddev_net(shared_features))
        )

        return action_means, action_stddevs

class Policy:
    """REINFORCE algorithm."""

    def __init__(self, obs_space_dims: int, action_space_dims: int, electromagnets, device="cpu"):
        """Initializes an agent that learns a policy via REINFORCE algorithm.
        Args:
            obs_space_dims: Dimension of the observation space
            action_space_dims: Dimension of the action space
        """
        self.action_space_dims = action_space_dims

        # Hyperparameters
        self.learning_rate = 5e-4  # Learning rate for policy optimization
        self.gamma = 0.99  # Discount factor
        self.eps = 1e-6  # small number for mathematical stability

        self.probs = []  # Stores probability values of the sampled action
        self.rewards = []  # Stores the corresponding rewards
        self.device = device
        self.electromag_pos = electromagnets
        self.net = Policy_Network(obs_space_dims, action_space_dims).to(device)
        self.optimizer = torch.optim.AdamW(self.net.parameters(), lr=self.learning_rate)

    def augment_obs(self, obs):
        # obs is 9x1 of XYZ (ball.position,ball.velocity,desired_position)
        ext_data = torch.zeros((1,1 + len(self.electromag_pos))).to(self.device)
        aug_obs = torch.cat((obs, ext_data), axis = 1)
        # Error
        aug_obs[0][9] = torch.linalg.norm(obs[0][6:9] - obs[0][:3])
        for i, mag_pos in enumerate(self.electromag_pos):
            # Distance to electromagnet
            aug_obs[0][10+i] = torch.linalg.norm(torch.tensor(mag_pos) - obs[0][:3])
        return aug_obs

    def sample_action(self, state: np.ndarray) -> float:
        """Returns action(s), conditioned on the policy and observation.

        Args:
            state: Observation from the environment nx1
        Returns:
            action: Action(s) to be performed
        """
        state = torch.tensor(np.array([state])).float().to(self.device)
        # state = torch.tensor(np.array([state]))
        state = self.augment_obs(state)
        action_means, action_stddevs = self.net(state)
        
        action_means = action_means.flatten()
        action_stddevs = action_stddevs.flatten()
        # create a normal distribution from the predicted
        #   mean and standard deviation and sample all actions action
        actions = np.zeros(self.action_space_dims)
        for action_dim in range(self.action_space_dims):
            distrib = Normal(action_means[action_dim] + self.eps, action_stddevs[action_dim] + self.eps)
            action = distrib.sample()
            prob = distrib.log_prob(action)
            actions[action_dim] = action

            self.probs.append(prob)

        return actions

    def update(self):
        """Updates the policy network's weights."""
        running_g = 0
        gs = []

        # Discounted return (backwards) - [::-1] will return an array in reverse
        for R in self.rewards[::-1]:
            running_g = R + self.gamma * running_g
            gs.insert(0, running_g)

        deltas = torch.tensor(gs).to(self.device)

        loss = 0
        # minimize -1 * prob * reward obtained
        for log_prob, delta in zip(self.probs, deltas):
            loss += log_prob.mean() * delta * (-1)

        # Update the policy network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Empty / zero out all episode-centric/related variables
        self.probs = []
        self.rewards = []

In [3]:
# device = ("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu" # current implementation has cpu being faster
print(f"Using {device} device") 

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

def generate_points_on_circle(radius, center, z, n):
    h, k = center
    angles = np.linspace(0, 2*np.pi, n, endpoint=False)  # Generate equally spaced angles
    x = h + radius * np.cos(angles)
    y = k + radius * np.sin(angles)
    z_values = np.full_like(x, z)  # Create an array filled with the z value
    points = np.column_stack((x, y, z_values))
    return points

# Example usage:
radius = 2
center = (0, 0)
num_points = 8

circle_points = generate_points_on_circle(radius, center, 0, num_points)
mag_coords = list(circle_points)
mag_coords.append(np.array([0,0,0]))

# mag_coords = [np.array([0,0,0]),]
spawn_range = ((-1.5,1.5),(-1.5,1.5),(2,3))
desired_range = ((-1.2,1.2),(-1.2,1.2),(2,2.8))
# Create and wrap the environment
env = MagneticEnv(mag_coords, DT, spawn_range, desired_range)
record_int = 10
wrapped_env = gym.wrappers.RecordEpisodeStatistics(env, record_int +1)  # Records episode-reward

# Reinitialize agent every seed
electro_positions = [e.position for e in env.electromagnets]

# extra dimensions for engineered features 
# 1 for error n for electromag distance
obs_space_dims = env.observation_space.shape[0] + 1 + len(electro_positions)
action_space_dims = env.action_space.shape[0]

Using cpu device


In [4]:
# matplotlib.rc_file_defaults()
# sns.reset_orig()
# plt.clf()

# env.fig = plt.figure()
# env.ax = env.fig.add_subplot(111, projection="3d")
# env.fig.tight_layout()
# plt.show()

In [5]:
# Reinitialize agent and record
agent = Policy(obs_space_dims, action_space_dims,electro_positions, device)
reward_over_episodes = []

In [6]:
DO_RENDER = True
total_num_episodes = int(75_000)  # Total number of episodes

#Tqdm progress bar object contains a list of the batch indices to train over
progress_bar = tqdm(range(total_num_episodes), desc='Training...', leave=False, disable=False)

for episode in progress_bar:
    obs, info = wrapped_env.reset(seed=RANDOM_SEED)
    done = False
    
    while not done:
        action = agent.sample_action(obs)
        obs, reward, terminated, truncated, info = wrapped_env.step(action)
        agent.rewards.append(reward)

        if DO_RENDER: wrapped_env.env.render()

        done = terminated or truncated

    if episode % record_int == 0:
        avg_reward = int(np.mean(wrapped_env.return_queue))
        reward_over_episodes.append(avg_reward)

    reward_val = wrapped_env.return_queue[-1]
    
    progress_bar.set_postfix({"Stats": f"Epoch: {episode}, Avg Reward: {avg_reward}, Reward {reward_val}"})
    agent.update()

Training...:   0%|          | 36/75000 [02:18<83:12:49,  4.00s/it, Stats=Epoch: 35, Avg Reward: -847, Reward [-1007.1406]] 

In [ ]:
from stable_baselines3 import SAC

env = MagneticEnv(mag_coords, DT)
record_int = 10
wrapped_env = gym.wrappers.RecordEpisodeStatistics(env, record_int +1)  # Records episode-reward

model = SAC("MlpPolicy", wrapped_env, verbose=1)
model.learn(total_timesteps=10000, log_interval=4, progress_bar=True)
obs, info = env.reset()


In [ ]:
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = wrapped_env.step(action)
    wrapped_env.env.render()
    if terminated or truncated:
        obs, info = env.reset()

In [ ]:
rewards_to_plot = np.array(reward_over_episodes)
xs = np.arange(len(rewards_to_plot)) * record_int

result_fig = plt.figure()
plt.clf()
result_ax = result_fig.add_subplot(111)

coef = np.polyfit(xs,rewards_to_plot,1)
result_ax.plot(xs, rewards_to_plot, label= "Rewards")
poly1d_fn = np.poly1d(coef) 

result_ax.plot(xs, poly1d_fn(xs), '--r', label= "Learning Loss" ) #'--k'=black dashed line, 'yo' = yellow circle marker
result_ax.set_xlabel("Episodes")
result_ax.set_ylabel("Reward")
result_ax.set_title("Levitation Learn")
result_ax.set_ylim([-12000,7000])

plt.show()